In [1]:
import numpy as np
import tensorflow as tf
import pandas as pd

In [2]:
from tensorflow.keras.layers import Dense

In [3]:
data=pd.read_csv("C:/Users/nchar/Desktop/english_to_french_translator/eng_-french.csv")
df=pd.DataFrame(data)
print(df.info())
df.dropna(axis=0,inplace=True)
inp=df["English words/sentences"][0:10000]
inp=list(inp)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 175621 entries, 0 to 175620
Data columns (total 2 columns):
 #   Column                   Non-Null Count   Dtype 
---  ------                   --------------   ----- 
 0   English words/sentences  175621 non-null  object
 1   French words/sentences   175621 non-null  object
dtypes: object(2)
memory usage: 2.7+ MB
None


In [4]:
inp=[(word.split()) for word in inp]

In [5]:
inp

[['Hi.'],
 ['Run!'],
 ['Run!'],
 ['Who?'],
 ['Wow!'],
 ['Fire!'],
 ['Help!'],
 ['Jump.'],
 ['Stop!'],
 ['Stop!'],
 ['Stop!'],
 ['Wait!'],
 ['Wait!'],
 ['Go', 'on.'],
 ['Go', 'on.'],
 ['Go', 'on.'],
 ['Hello!'],
 ['Hello!'],
 ['I', 'see.'],
 ['I', 'try.'],
 ['I', 'won!'],
 ['I', 'won!'],
 ['I', 'won.'],
 ['Oh', 'no!'],
 ['Attack!'],
 ['Attack!'],
 ['Cheers!'],
 ['Cheers!'],
 ['Cheers!'],
 ['Cheers!'],
 ['Get', 'up.'],
 ['Go', 'now.'],
 ['Go', 'now.'],
 ['Go', 'now.'],
 ['Got', 'it!'],
 ['Got', 'it!'],
 ['Got', 'it?'],
 ['Got', 'it?'],
 ['Got', 'it?'],
 ['Hop', 'in.'],
 ['Hop', 'in.'],
 ['Hug', 'me.'],
 ['Hug', 'me.'],
 ['I', 'fell.'],
 ['I', 'fell.'],
 ['I', 'know.'],
 ['I', 'left.'],
 ['I', 'left.'],
 ['I', 'lied.'],
 ['I', 'lost.'],
 ['I', 'paid.'],
 ["I'm", '19.'],
 ["I'm", 'OK.'],
 ["I'm", 'OK.'],
 ['Listen.'],
 ['No', 'way!'],
 ['No', 'way!'],
 ['No', 'way!'],
 ['No', 'way!'],
 ['No', 'way!'],
 ['No', 'way!'],
 ['No', 'way!'],
 ['No', 'way!'],
 ['No', 'way!'],
 ['Really?'],
 ['Real

In [6]:
t=tf.keras.preprocessing.text.Tokenizer()
t.fit_on_texts(inp)

In [7]:
inp_encds=t.texts_to_sequences(inp)
inp_encds

[[1262],
 [1263],
 [1263],
 [1264],
 [1946],
 [1265],
 [1947],
 [975],
 [976],
 [976],
 [976],
 [1266],
 [1266],
 [35, 202],
 [35, 202],
 [35, 202],
 [1267],
 [1267],
 [1, 450],
 [1, 190],
 [1, 792],
 [1, 792],
 [1, 154],
 [977, 1948],
 [1268],
 [1268],
 [793],
 [793],
 [793],
 [793],
 [33, 41],
 [35, 94],
 [35, 94],
 [35, 94],
 [46, 106],
 [46, 106],
 [46, 51],
 [46, 51],
 [46, 51],
 [1269, 100],
 [1269, 100],
 [978, 24],
 [978, 24],
 [1, 794],
 [1, 794],
 [1, 179],
 [1, 191],
 [1, 191],
 [1, 675],
 [1, 112],
 [1, 979],
 [2, 1949],
 [2, 149],
 [2, 149],
 [1270],
 [37, 351],
 [37, 351],
 [37, 351],
 [37, 351],
 [37, 351],
 [37, 351],
 [37, 351],
 [37, 351],
 [37, 351],
 [795],
 [795],
 [795],
 [500],
 [7, 190],
 [7, 154],
 [7, 154],
 [7, 154],
 [7, 154],
 [123, 47],
 [1950],
 [15, 278],
 [15, 278],
 [15, 278],
 [15, 192],
 [15, 253],
 [15, 253],
 [15, 253],
 [15, 253],
 [15, 253],
 [15, 253],
 [15, 580],
 [15, 193],
 [15, 193],
 [15, 193],
 [15, 193],
 [15, 193],
 [15, 193],
 [796, 12]

In [8]:
m=max([len(text) for text in inp_encds])
print(m)
m=max([len(text) for text in inp_encds])
def zero_padding_and_masking(sequences,max_len):
    batch_size=len(sequences)
    padded_sequences=np.zeros((batch_size,max_len),dtype=np.int32)
    mask=np.zeros((batch_size,max_len))
    for i,seq in enumerate(sequences):
        seq_len=len(seq)
        padded_sequences[i,:seq_len]=seq
        mask[i,:seq_len]=1
    mask[mask==0]=-np.infty
    mask[mask==1]=0
    mask=tf.reshape(mask,(batch_size,1,max_len)) # here we mask shape is (batch_size,seq_len) we reshape it to (batch_size,1,seq_len) or (100000,1,10)  because 
    # the shape of look_ahead_mask will be (batch_size,seq_len,seq_len) or (10000,10,10).hence reshape the padding_mask so that during addition of both array the padding mask will added perfectly with the help of broadcastting
    mask=tf.broadcast_to(mask,(10000,10,10))
    mask=tf.expand_dims(mask,axis=1) # here we expanded mask along the dimension 1 so it will be converted to (batch_size,1,seq_len,seq_len).we do this because when we add the mask to the qk/dk or attention weights the size of attention weights is (batch_size,num_heads,seq_len,seq_len

    return padded_sequences,mask
        
        
        
    
    
inp_encds,padding_mask=zero_padding_and_masking(inp_encds,10)
inp_encds.shape
    

5


(10000, 10)

In [9]:
inp_encds

array([[1262,    0,    0, ...,    0,    0,    0],
       [1263,    0,    0, ...,    0,    0,    0],
       [1263,    0,    0, ...,    0,    0,    0],
       ...,
       [  15,   99,  163, ...,    0,    0,    0],
       [  15,   99,  163, ...,    0,    0,    0],
       [  15,   99,  163, ...,    0,    0,    0]])

In [10]:
padding_mask

<tf.Tensor: shape=(10000, 1, 10, 10), dtype=float64, numpy=
array([[[[  0., -inf, -inf, ..., -inf, -inf, -inf],
         [  0., -inf, -inf, ..., -inf, -inf, -inf],
         [  0., -inf, -inf, ..., -inf, -inf, -inf],
         ...,
         [  0., -inf, -inf, ..., -inf, -inf, -inf],
         [  0., -inf, -inf, ..., -inf, -inf, -inf],
         [  0., -inf, -inf, ..., -inf, -inf, -inf]]],


       [[[  0., -inf, -inf, ..., -inf, -inf, -inf],
         [  0., -inf, -inf, ..., -inf, -inf, -inf],
         [  0., -inf, -inf, ..., -inf, -inf, -inf],
         ...,
         [  0., -inf, -inf, ..., -inf, -inf, -inf],
         [  0., -inf, -inf, ..., -inf, -inf, -inf],
         [  0., -inf, -inf, ..., -inf, -inf, -inf]]],


       [[[  0., -inf, -inf, ..., -inf, -inf, -inf],
         [  0., -inf, -inf, ..., -inf, -inf, -inf],
         [  0., -inf, -inf, ..., -inf, -inf, -inf],
         ...,
         [  0., -inf, -inf, ..., -inf, -inf, -inf],
         [  0., -inf, -inf, ..., -inf, -inf, -inf],
      

In [11]:
len(t.word_index)

2946

In [12]:
embedding_dim=d_model=128
vocab_size=10000
embedding_layer=tf.keras.layers.Embedding(vocab_size,embedding_dim)
embedded_tokens=embedding_layer(inp_encds)
print(embedded_tokens.shape)

(10000, 10, 128)


In [13]:
embedded_tokens

<tf.Tensor: shape=(10000, 10, 128), dtype=float32, numpy=
array([[[-0.04533749,  0.02042717, -0.03344303, ...,  0.0328843 ,
         -0.03221156,  0.01272133],
        [-0.03218901,  0.0384547 ,  0.04832992, ..., -0.00894828,
          0.01788006,  0.03959267],
        [-0.03218901,  0.0384547 ,  0.04832992, ..., -0.00894828,
          0.01788006,  0.03959267],
        ...,
        [-0.03218901,  0.0384547 ,  0.04832992, ..., -0.00894828,
          0.01788006,  0.03959267],
        [-0.03218901,  0.0384547 ,  0.04832992, ..., -0.00894828,
          0.01788006,  0.03959267],
        [-0.03218901,  0.0384547 ,  0.04832992, ..., -0.00894828,
          0.01788006,  0.03959267]],

       [[ 0.0333528 , -0.03671128,  0.0290984 , ..., -0.03828328,
         -0.01030395,  0.0439488 ],
        [-0.03218901,  0.0384547 ,  0.04832992, ..., -0.00894828,
          0.01788006,  0.03959267],
        [-0.03218901,  0.0384547 ,  0.04832992, ..., -0.00894828,
          0.01788006,  0.03959267],
        .

In [14]:
num_heads=8
seq_len=10
dim_head=embedding_dim//num_heads
dim_head

16

In [15]:
class MultiHeadAttention(tf.keras.layers.Layer):
    def __init__(self,d_model:int,num_heads:int) -> None :
        super().__init__()
        self.d_model=d_model
        self.num_heads=num_heads
        self.wq=Dense(d_model)
        self.wk=Dense(d_model)
        self.wv=Dense(d_model)
        self.dense=Dense(d_model)
        self.d_head=d_model//num_heads

    
    def scaled_dot_product_attention(self,q,k,v,mask):
        a=tf.matmul(q,k,transpose_b=True)
        dk = tf.cast(tf.shape(k)[-1], tf.float32)
        attention_weights=a/tf.math.sqrt(dk)
        if mask is not None:
            attention_weights +=mask
        attention_scores=tf.nn.softmax(attention_weights)
        outputs=tf.matmul(attention_scores,v)
        return outputs,attention_scores

    
    def split_heads(self, x, batch_size,seq_len):
        x = tf.reshape(x, (batch_size,seq_len, self.num_heads, self.d_head))
        return tf.transpose(x, perm=[0, 2, 1, 3])

    
    def call(self,x,mask):
        batch_size=tf.shape(x)[0]
        seq_len=tf.shape(x)[1]
        
        q=self.wq(x)
        k=self.wk(x)
        v=self.wv(x)
        q=self.split_heads(q,batch_size,seq_len)
        k=self.split_heads(k,batch_size,seq_len)
        v=self.split_heads(v,batch_size,seq_len)
        outputs,attention_weights=self.scaled_dot_product_attention(q,k,v,mask)
        attention = tf.transpose(outputs, perm=[0, 2, 1, 3])
        attention = tf.reshape(attention, (batch_size, -1, self.d_model))
        output = self.dense(attention)
        return output

# Instantiate the MultiHeadAttention layer

# Example number of attention heads
mha_layer = MultiHeadAttention(d_model, num_heads)

# Call the layer
output = mha_layer(embedded_tokens,mask=padding_mask)
output

# Now `output` will be the result of the multi-head attention operation


<tf.Tensor: shape=(10000, 10, 128), dtype=float32, numpy=
array([[[-0.04676005,  0.0337153 ,  0.04421903, ...,  0.01074874,
          0.00635769, -0.00952436],
        [-0.04676005,  0.0337153 ,  0.04421903, ...,  0.01074874,
          0.00635769, -0.00952436],
        [-0.04676005,  0.0337153 ,  0.04421903, ...,  0.01074874,
          0.00635769, -0.00952436],
        ...,
        [-0.04676005,  0.0337153 ,  0.04421903, ...,  0.01074874,
          0.00635769, -0.00952436],
        [-0.04676005,  0.0337153 ,  0.04421903, ...,  0.01074874,
          0.00635769, -0.00952436],
        [-0.04676005,  0.0337153 ,  0.04421903, ...,  0.01074874,
          0.00635769, -0.00952436]],

       [[ 0.04112805,  0.05105   ,  0.01866102, ...,  0.00375406,
          0.01497328,  0.00043063],
        [ 0.04112805,  0.05105   ,  0.01866102, ...,  0.00375406,
          0.01497328,  0.00043063],
        [ 0.04112805,  0.05105   ,  0.01866102, ...,  0.00375406,
          0.01497328,  0.00043063],
        .

In [16]:
mha_layer.trainable_variables

[<KerasVariable shape=(128, 128), dtype=float32, path=multi_head_attention/dense/kernel>,
 <KerasVariable shape=(128,), dtype=float32, path=multi_head_attention/dense/bias>,
 <KerasVariable shape=(128, 128), dtype=float32, path=multi_head_attention/dense_1/kernel>,
 <KerasVariable shape=(128,), dtype=float32, path=multi_head_attention/dense_1/bias>,
 <KerasVariable shape=(128, 128), dtype=float32, path=multi_head_attention/dense_2/kernel>,
 <KerasVariable shape=(128,), dtype=float32, path=multi_head_attention/dense_2/bias>,
 <KerasVariable shape=(128, 128), dtype=float32, path=multi_head_attention/dense_3/kernel>,
 <KerasVariable shape=(128,), dtype=float32, path=multi_head_attention/dense_3/bias>]

In [17]:
class MultiHeadAttentionWithoutMask(tf.keras.layers.Layer):
    def __init__(self,d_model:int,num_heads:int) -> None :
        super().__init__()
        self.d_model=d_model
        self.num_heads=num_heads
        self.wq=Dense(d_model)
        self.wk=Dense(d_model)
        self.wv=Dense(d_model)
        self.dense=Dense(d_model)
        self.d_head=d_model//num_heads

    
    def scaled_dot_product_attention(self,q,k,v):
        a=tf.matmul(q,k,transpose_b=True)
        dk = tf.cast(tf.shape(k)[-1], tf.float32)
        attention_weights=a/tf.math.sqrt(dk)
        #if mask is not None:
            #attention_weights +=mask
        attention_scores=tf.nn.softmax(attention_weights)
        outputs=tf.matmul(attention_scores,v)
        return outputs,attention_scores

    
    def split_heads(self, x, batch_size,seq_len):
        x = tf.reshape(x, (batch_size,seq_len, self.num_heads, self.d_head))
        return tf.transpose(x, perm=[0, 2, 1, 3])

    
    def call(self,x):
        batch_size=tf.shape(x)[0]
        seq_len=tf.shape(x)[1]
        
        q=self.wq(x)
        k=self.wk(x)
        v=self.wv(x)
        q=self.split_heads(q,batch_size,seq_len)
        k=self.split_heads(k,batch_size,seq_len)
        v=self.split_heads(v,batch_size,seq_len)
        outputs,attention_weights=self.scaled_dot_product_attention(q,k,v)
        attention = tf.transpose(outputs, perm=[0, 2, 1, 3])
        attention = tf.reshape(attention, (batch_size, -1, self.d_model))
        output = self.dense(attention)
        return output

# Instantiate the MultiHeadAttention layer

# Example number of attention heads
mha_layer = MultiHeadAttentionWithoutMask(d_model, num_heads)

# Call the layer
output = mha_layer(embedded_tokens)
output

# Now `output` will be the result of the multi-head attention operation


<tf.Tensor: shape=(10000, 10, 128), dtype=float32, numpy=
array([[[ 0.0046807 ,  0.01768465,  0.00134859, ..., -0.01141886,
         -0.0385076 , -0.01529707],
        [ 0.00467924,  0.01768328,  0.00134976, ..., -0.01140843,
         -0.03851391, -0.01529734],
        [ 0.00467924,  0.01768328,  0.00134976, ..., -0.01140843,
         -0.03851391, -0.01529734],
        ...,
        [ 0.00467924,  0.01768328,  0.00134976, ..., -0.01140843,
         -0.03851391, -0.01529734],
        [ 0.00467924,  0.01768328,  0.00134976, ..., -0.01140843,
         -0.03851391, -0.01529734],
        [ 0.00467924,  0.01768328,  0.00134976, ..., -0.01140843,
         -0.03851391, -0.01529734]],

       [[ 0.00335277,  0.01327814,  0.00445145, ..., -0.01188496,
         -0.03670066, -0.01697137],
        [ 0.00336073,  0.0132715 ,  0.00445382, ..., -0.01188272,
         -0.03670301, -0.01696932],
        [ 0.00336073,  0.0132715 ,  0.00445382, ..., -0.01188272,
         -0.03670301, -0.01696932],
        .

In [18]:
class Feedforward(tf.keras.layers.Layer):
    def __init__(self,d_model,dff):
        super().__init__()
        self.d_model=d_model
        self.dff=dff
        self.layer1=tf.keras.layers.Dense(dff,activation="relu")
        self.layer2=tf.keras.layers.Dense(d_model)
    def call(self,x):
        out1=self.layer1(x)
        out2=self.layer2(out1)
        return out2

In [19]:
class Encoder_layer(tf.keras.layers.Layer):
    def __init__(self,d_model,num_heads,dff):
        super().__init__()
        self.attention=MultiHeadAttention(d_model=d_model,num_heads=num_heads)
        self.norm_and_add1=tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.norm_and_add2=tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.feed_forward=Feedforward(dff=dff,d_model=d_model)
    def call(self,x,mask):
        attention_out=self.attention(x,mask)
        norm_and_add1_out=self.norm_and_add1(x+attention_out)
        feed_forward_out=self.feed_forward(norm_and_add1_out)
        norm_and_add2_out=self.norm_and_add2(norm_and_add1_out+feed_forward_out)
        return norm_and_add2_out

In [20]:
s=Encoder_layer(d_model=128,num_heads=8,dff=1024)
out=s(embedded_tokens,padding_mask)
out

<tf.Tensor: shape=(10000, 10, 128), dtype=float32, numpy=
array([[[-1.4886968 ,  0.11321091, -0.97831094, ..., -0.5583615 ,
          0.15845537, -0.54147947],
        [-1.1725419 ,  0.2549905 ,  0.90071696, ..., -1.000593  ,
          1.1341578 ,  0.28369924],
        [-1.1725419 ,  0.2549905 ,  0.90071696, ..., -1.000593  ,
          1.1341578 ,  0.28369924],
        ...,
        [-1.1725419 ,  0.2549905 ,  0.90071696, ..., -1.000593  ,
          1.1341578 ,  0.28369924],
        [-1.1725419 ,  0.2549905 ,  0.90071696, ..., -1.000593  ,
          1.1341578 ,  0.28369924],
        [-1.1725419 ,  0.2549905 ,  0.90071696, ..., -1.000593  ,
          1.1341578 ,  0.28369924]],

       [[ 0.594838  , -0.07899118,  1.2215734 , ..., -0.6402409 ,
         -1.5130701 ,  0.9455099 ],
        [-1.5793152 ,  1.2998207 ,  1.5774137 , ...,  0.22933276,
         -0.9202835 ,  0.80294514],
        [-1.5793152 ,  1.2998207 ,  1.5774137 , ...,  0.22933276,
         -0.9202835 ,  0.80294514],
        .

In [21]:
class Encoder_layerWithoutMask(tf.keras.layers.Layer):
    def __init__(self,d_model,num_heads,dff):
        super().__init__()
        self.attention=MultiHeadAttentionWithoutMask(d_model=d_model,num_heads=num_heads)
        self.norm_and_add1=tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.norm_and_add2=tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.feed_forward=Feedforward(dff=dff,d_model=d_model)
    def call(self,x):
        attention_out=self.attention(x)
        norm_and_add1_out=self.norm_and_add1(x+attention_out)
        feed_forward_out=self.feed_forward(norm_and_add1_out)
        norm_and_add2_out=self.norm_and_add2(norm_and_add1_out+feed_forward_out)
        return norm_and_add2_out

In [22]:
class SequentialEncoder(tf.keras.layers.Layer):
    def __init__(self, layers):
        super().__init__()
        self.layers = layers

    def call(self, x,mask):
        for layer in self.layers:
            x = layer(x,mask)
        return x

"WE CAN ALSO USE TENSORFLOW SEQUENTIAL MODULE TO IMPLEMENT MULTIPLE ENCODER_LAYER BUT BY USING THIS WE CAN ONLY GIVE ONE INPUT FOR THE CALL FUNCTION OF ENCODER_LAYER,BUT WE HAVE TO GIVE X AND THE MASK FOR THE CALL FUNCTION SO WE CREATE A SEQUENTIAL_ENCODER CLASS TO IMPLEMENT THIS

self.layer=tf.keras.models.Sequential([Encoder_layer(d_model,num_heads,dff) for _ in range(self.num_encoders)])
    def call(self,x):
        x=self.layer(x)
        return x
    

In [23]:
class EncoderWithoutMask(tf.keras.layers.Layer):
    def __init__(self,d_model,num_heads,dff,num_encoders):
        super().__init__()
        self.num_encoders=num_encoders
        self.layer=tf.keras.models.Sequential([Encoder_layerWithoutMask(d_model,num_heads,dff) for _ in range(self.num_encoders)])
    def call(self,x):
        x=self.layer(x)
        return x
    

In [24]:
class Encoder(tf.keras.layers.Layer):
    def __init__(self, d_model,num_heads,dff,num_encoders):
        super().__init__()
        self.layers = SequentialEncoder([Encoder_layer(d_model=d_model,num_heads=num_heads,dff=dff) for _ in range(num_encoders)])

    def call(self, x,mask):
        x = self.layers(x,mask)
        return x

In [25]:
o=Encoder(d_model=128,num_heads=8,dff=512,num_encoders=12)
d=o(embedded_tokens,mask=padding_mask)
d

<tf.Tensor: shape=(10000, 10, 128), dtype=float32, numpy=
array([[[ 1.0869293 , -0.38378254, -1.1839347 , ..., -0.5791592 ,
          0.5229432 , -0.30214807],
        [ 1.1010612 , -0.38273725, -1.1493719 , ..., -0.5805572 ,
          0.5687945 , -0.28164622],
        [ 1.1010612 , -0.38273725, -1.1493719 , ..., -0.5805572 ,
          0.5687945 , -0.28164622],
        ...,
        [ 1.1010612 , -0.38273725, -1.1493719 , ..., -0.5805572 ,
          0.5687945 , -0.28164622],
        [ 1.1010612 , -0.38273725, -1.1493719 , ..., -0.5805572 ,
          0.5687945 , -0.28164622],
        [ 1.1010612 , -0.38273725, -1.1493719 , ..., -0.5805572 ,
          0.5687945 , -0.28164622]],

       [[ 1.3873398 ,  0.8867936 , -0.22183435, ..., -0.38528258,
         -0.35816532,  0.4737211 ],
        [ 1.3821946 ,  0.90449274, -0.24043608, ..., -0.3720818 ,
         -0.36139715,  0.48589143],
        [ 1.3821946 ,  0.90449274, -0.24043608, ..., -0.3720818 ,
         -0.36139715,  0.48589143],
        .

In [26]:
o.trainable_variables

[<KerasVariable shape=(128, 128), dtype=float32, path=encoder/sequential_encoder/encoder_layer_1/multi_head_attention_2/dense_14/kernel>,
 <KerasVariable shape=(128,), dtype=float32, path=encoder/sequential_encoder/encoder_layer_1/multi_head_attention_2/dense_14/bias>,
 <KerasVariable shape=(128, 128), dtype=float32, path=encoder/sequential_encoder/encoder_layer_1/multi_head_attention_2/dense_15/kernel>,
 <KerasVariable shape=(128,), dtype=float32, path=encoder/sequential_encoder/encoder_layer_1/multi_head_attention_2/dense_15/bias>,
 <KerasVariable shape=(128, 128), dtype=float32, path=encoder/sequential_encoder/encoder_layer_1/multi_head_attention_2/dense_16/kernel>,
 <KerasVariable shape=(128,), dtype=float32, path=encoder/sequential_encoder/encoder_layer_1/multi_head_attention_2/dense_16/bias>,
 <KerasVariable shape=(128, 128), dtype=float32, path=encoder/sequential_encoder/encoder_layer_1/multi_head_attention_2/dense_17/kernel>,
 <KerasVariable shape=(128,), dtype=float32, path=en

In [27]:
len(o.trainable_variables)

192

"THERE ARE ANOTHER TYPES TO IMPLEMENT THE ENCODER THEY ARE":


self.layers = tf.keras.layers.LayerList([ Encoder_layer(seq_len, d_model, num_heads, dff) for _ in range(self.num_encoders) ])
def call(self, x, training=False):
    for layer in self.layers:
        x = layer(x, training=training)
    return x

and  another one will be:

self.layers=[ Encoder_layer(seq_len, d_model, num_heads, dff) for _ in range(self.num_encoders) ]
def call(self, x, training=False):
        for i in range(self.num_encoders) :
            x=self.layers[i](x)
        return x

In [28]:
a=EncoderWithoutMask(d_model=128,num_heads=8,dff=512,num_encoders=12)
b=a(embedded_tokens)
b

<tf.Tensor: shape=(10000, 10, 128), dtype=float32, numpy=
array([[[-1.0224077 ,  0.18520403,  1.8132759 , ...,  0.4453954 ,
          0.9375175 ,  0.04725852],
        [-1.0209213 ,  0.18806212,  1.8311243 , ...,  0.4497649 ,
          0.9583252 ,  0.03651921],
        [-1.0209213 ,  0.18806212,  1.8311243 , ...,  0.4497649 ,
          0.9583252 ,  0.03651921],
        ...,
        [-1.0209213 ,  0.18806212,  1.8311243 , ...,  0.4497649 ,
          0.9583252 ,  0.03651921],
        [-1.0209213 ,  0.18806212,  1.8311243 , ...,  0.4497649 ,
          0.9583252 ,  0.03651921],
        [-1.0209213 ,  0.18806212,  1.8311243 , ...,  0.4497649 ,
          0.9583252 ,  0.03651921]],

       [[-0.9847155 ,  0.09940192,  1.7909527 , ...,  0.40147868,
          0.8218212 ,  0.00733665],
        [-0.9724947 ,  0.10230605,  1.7717413 , ...,  0.41143274,
          0.84681004, -0.00926694],
        [-0.9724947 ,  0.10230605,  1.7717413 , ...,  0.41143274,
          0.84681004, -0.00926694],
        .

In [29]:
a.trainable_variables

[<KerasVariable shape=(128, 128), dtype=float32, path=encoder_without_mask/sequential/encoder_layer_without_mask/multi_head_attention_without_mask_1/dense_86/kernel>,
 <KerasVariable shape=(128,), dtype=float32, path=encoder_without_mask/sequential/encoder_layer_without_mask/multi_head_attention_without_mask_1/dense_86/bias>,
 <KerasVariable shape=(128, 128), dtype=float32, path=encoder_without_mask/sequential/encoder_layer_without_mask/multi_head_attention_without_mask_1/dense_87/kernel>,
 <KerasVariable shape=(128,), dtype=float32, path=encoder_without_mask/sequential/encoder_layer_without_mask/multi_head_attention_without_mask_1/dense_87/bias>,
 <KerasVariable shape=(128, 128), dtype=float32, path=encoder_without_mask/sequential/encoder_layer_without_mask/multi_head_attention_without_mask_1/dense_88/kernel>,
 <KerasVariable shape=(128,), dtype=float32, path=encoder_without_mask/sequential/encoder_layer_without_mask/multi_head_attention_without_mask_1/dense_88/bias>,
 <KerasVariable 

In [30]:
len(a.trainable_variables)

192